In [13]:
import tensorflow as tf

In [17]:
tf.reset_default_graph()
cluster_spec = tf.train.ClusterSpec({'ps' : ['localhost:2223'],'worker' : ['localhost:2224']})

Create global variables

In [18]:
with tf.device(tf.train.replica_device_setter(ps_tasks=1,cluster=cluster_spec)):
    a = tf.Variable([0.],name='a')
    b = tf.constant([100.])
    loss = tf.abs(a-b)
    
    opt = tf.train.GradientDescentOptimizer(.1).minimize(loss)
    

Notice that the variables are on the parameters server

In [19]:
for v in tf.trainable_variables():
    print v.device

/job:ps/task:0


Now create the local copies

In [20]:
#with tf.device('/job:worker/task:0'): #do not need to explicityly state the device with local anchor!
with tf.device(tf.train.replica_device_setter(ps_tasks=1,cluster=cluster_spec)):
    local_anchor = tf.no_op()
    with tf.colocate_with(local_anchor):
        for v in tf.global_variables() :
            v_l = tf.get_variable(name='local/%s'%(v.op.name),
                initializer=[0.],
                trainable=True,
                collections=[tf.GraphKeys.LOCAL_VARIABLES])

In [6]:
with tf.device(tf.train.replica_device_setter(ps_tasks=1,cluster=cluster_spec)):    
    init = tf.global_variables_initializer()
    init_local = tf.local_variables_initializer()

In [7]:
for v in tf.local_variables():
    print v.name,v.device

local/a:0 /job:worker


In [8]:
server = tf.train.Server(cluster_spec,job_name='worker')
sess = tf.Session(target=server.target)

In [9]:
sess.run(init)

In [10]:
a_l = [v for v in tf.local_variables() if 'local/a:0' == v.name ][0]

In [11]:
#sess.run([a_l]) #will through error
sess.run(a)

array([ 0.], dtype=float32)

In [12]:
sess.run(init_local)
sess.run(a_l)

array([ 0.], dtype=float32)